In [1]:
import requests
from bs4 import BeautifulSoup
import html2text
import mygene
import json
import pickle
mg = mygene.MyGeneInfo()

In [33]:
response = requests.get(f"https://www.ncbi.nlm.nih.gov/datasets/gene/100133941/", timeout=30)

In [34]:
soup = BeautifulSoup(response.content, 'html.parser')

In [35]:
soup.contents

['html',
 '\n',
 <html lang="en">
 <head>
 <meta charset="utf-8"/>
 <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
 <meta content="True" name="HandheldFriendly"/>
 <meta content="320" name="MobileOptimized"/>
 <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
 <!-- UA-68648221-3 -->
 <!-- Favicons -->
 <link href="https://www.ncbi.nlm.nih.gov/coreutils/nwds/img/favicons/favicon.ico" rel="shortcut icon" type="image/ico"/>
 <link href="https://www.ncbi.nlm.nih.gov/coreutils/nwds/img/favicons/favicon.png" rel="icon" type="image/png"/>
 <!-- 192x192, as recommended for Android
 http://updates.html5rocks.com/2014/11/Support-for-theme-color-in-Chrome-39-for-Android
 -->
 <link href="https://www.ncbi.nlm.nih.gov/coreutils/nwds/img/favicons/favicon-192.png" rel="icon" sizes="192x192" type="image/png"/>
 <!-- 57x57 (precomposed) for iPhone 3GS, pre-2011 iPod Touch and older Android devices -->
 <link href="https://www.ncbi.nlm.nih.gov/coreutils/nwds/img/favicons/fa

In [ ]:

summary_tab = soup.find('dl', recursive=True)
res = [""] * 3
# gene_type = "This gene is a " + summary_tab[9].contents[0] + ". "
# summary = summary_tab[19].contents[0].split("[")[0]
# expressed = summary_tab[21].contents[0] + ". "

# full_summary = gene_type + summary + expressed
# print(full_summary)

#print(len(summary_tab))
for i in range(0, len(summary_tab), 2):
    dt_tag = summary_tab[i]
    dd_tag = summary_tab[i+1]
    if dt_tag.name == 'dt':
        inner_str = dt_tag.string
        if inner_str == "Gene type":
            print(i)
            res[0] = "This gene is a " + dd_tag.contents[0] + "."
            continue
        elif inner_str == "Refseq summary":
            print(i)
            res[1] = dd_tag.contents[0].split("[")[0]
            continue
        elif inner_str == "Expression":
            print(i)
            res[2] = dd_tag.contents[0] + "."
            break

full_summary = " ".join(res)
print(f" summary: {full_summary}")

TypeError: object of type 'NoneType' has no len()

In [ ]:
parts_to_remove = [
    "##  Summary\n",
    "NEW",
    'Try the newGene table',
    'Try the newTranscript table',
    '**',
    "\nGo to the top of the page Help\n"
]

def rough_text_from_gene_name(gene_number):
    
    # get url
    url = f"https://www.ncbi.nlm.nih.gov/gene/{gene_number}"
    # Send a GET request to the URL
    summary_text = ''
    soup = None
    try:
        response = requests.get(url, timeout=30)
    except requests.exceptions.Timeout:
        print('time out')
        return((summary_text,soup))
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the "summary" tab content by inspecting the page's structure
        summary_tab = soup.find('div', {'class': 'rprt-section gene-summary'})

        # Check if the "summary" tab content is found
        if summary_tab:
            # Convert the HTML to plain text using html2text
            html_to_text = html2text.HTML2Text()
            html_to_text.ignore_links = True  # Ignore hyperlinks

            # Extract the plain text from the "summary" tab
            summary_text = html_to_text.handle(str(summary_tab))
            # Remove the specified parts from the original text
            for part in parts_to_remove:
                summary_text = summary_text.replace(part, ' ')
                # Replace '\n' with a space
            summary_text = summary_text.replace('\n', ' ')

            # Reduce multiple spaces into one space
            summary_text = ' '.join(summary_text.split())
            # Print or save the extracted text
            print(summary_text)
        else:
            print("Summary tab not found on the page.")
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
    return((summary_text,soup))

In [2]:
# Using Gene CD24 as an example
cd_24_name = mg.querymany('CD24', scopes='symbol', species='human')

Input sequence provided is already in string format. No operation performed


In [10]:
gene_name_to_tax_id = {}
for result in cd_24_name:
    if "_id" in result and "query" in result:
        gene_name_to_tax_id[result['symbol']] = result['_id']

In [5]:
gene_name_to_tax_id

{'CD24': '100133941'}

### Fetching Genes summaries

In [35]:
with open('/Users/maxwelladorsoo/Desktop/procupine/GenePT/input_data/vocab.json', 'rb') as handle:
    vocab_gene = json.load(handle)
vocab_gene_list = list(vocab_gene.keys())





In [36]:
## Fetch page ids
vocab_mg_query_result = mg.querymany(vocab_gene_list, scopes='symbol', species='human')

Input sequence provided is already in string format. No operation performed
6963 input query terms found dup hits:	[('PDCD6-AHRR', 2), ('DUX4L8', 3), ('KMT5AP1', 3), ('PTPN20CP', 2), ('NPEPPSP1', 2), ('SINHCAFP2', 2
22011 input query terms found no hit:	['RP5-973N23.5', 'RP11-182N22.10', 'CTB-53D8.3', 'RP11-348N17.2', 'RP11-205M20.8', 'RP11-326C3.17', 


In [37]:
## Convert query result to gene_name_to_page ids
gene_name_to_tax_id = {}
for result in vocab_mg_query_result:
    if "_id" in result and "query" in result:
        gene_name_to_tax_id[result['symbol']] = result['_id']

In [38]:
len(gene_name_to_tax_id)

38686

In [44]:
file_content = ""
for key, val in gene_name_to_tax_id.items():
    file_content += f"{key.strip()}\t{val.strip()}\n"

In [45]:
file_content[: 100]

'GET1-SH3BGR\t106865373\nTMX2-CTNND1\t100528016\nPDCD6-AHRR\t116412618\nDUX4L8\t26583\nDUX4L3\t653548\nBCLAF1P2'

In [43]:
with open("/Users/maxwelladorsoo/Desktop/procupine/GenePT/input_data/gene_list.txt", "w") as fh:
    fh.write(file_content)

In [13]:
gene_name_to_summary_page = {}

In [ ]:
for gene_name, page_id in sorted(gene_name_to_tax_id.items()):
    if gene_name not in gene_name_to_summary_page:
        print('gene_name',gene_name)
        parsed_text, unparsed_html = rough_text_from_gene_name(page_id)
        gene_name_to_summary_page[gene_name] = parsed_text

In [ ]:
gene_name_to_summary_page
json.dump("../input_data/gene_summaries.json")

### Experiment with Varied Gene Sets Tailored to Your Needs
#### For any specific gene name, mygene can be utilized to translate it into page IDs. We've illustrated an example using gene vocabularies in scGPT and Geneformer. Download links for these files are available in the repository's README.

In [ ]:
# load genes used in GenePT
with open(f"{data_dir}/vocab.json', 'rb') as handle:
    vocab_gene = json.load(handle)
vocab_gene_list = list(vocab_gene.keys())

# load genes used in Geneformer
with open(f"{data_dir}/token_dictionary.pkl", 'rb') as handle:
    token_dictionary = pickle.load(handle)

# example query to convert gene IDs into page ids for NCBI 
vocab_gene_list_results = mg.querymany(sorted(vocab_gene_list), scopes='symbol', species='human')
token_dictionary_results = mg.querymany(sorted(token_dictionary.keys()), fields="symbol")

In [1]:
import os
import pickle

In [6]:
with open("../gene_embeddings/biogpt_embeddings.pickle", 'rb') as f:
    gene_emb = pickle.load(f)

genes = os.listdir("../gene_summaries")
genes = [gene.split(".txt")[0] for gene in genes if gene.endswith(".txt")]
print(f"length: {len(genes)}")

for gene in genes:
    if gene in gene_emb:
        print(f"{gene} {gene_emb[gene][: 10]}")
   

length: 22890
A1BG-AS1 [-0.7038004   1.4458723   0.46534282  0.79608595 -1.2182738  -0.77830786
  0.5930708   0.551052   -0.83736026  1.3939788 ]
A1BG [-0.42330393  1.2548546   0.01991293  0.33651528 -1.4567515  -0.9367283
  0.9401818   0.53146905 -0.99964786  0.7632141 ]
A1CF [-0.4805924   1.2563351  -0.08980042  0.17618334 -1.2635323  -0.89650744
  0.73676115  0.70302325 -1.1107479   0.8642614 ]
A2M [ 0.3084071   1.547596   -0.11570039 -0.2073669  -0.85785466 -0.18636137
 -0.11323792  0.54637456 -0.5614272   0.60719806]
A2ML1 [-0.09966061  1.4991506   0.08268692 -1.0870228   0.07307071 -0.6045589
 -0.2875715   0.37510338 -0.6999969   0.32562166]
A2MP1 [-0.06261098  1.6177403   0.02481641 -1.1032155  -0.22602767 -0.6310245
 -0.25196618  0.49934316 -0.6960301   0.2066948 ]
A3GALT2 [ 0.3403953   1.0925328   0.3682044  -0.2804819  -0.34434104  0.0712922
  0.5151764  -0.735621   -2.333765    0.5393237 ]
A4GALT [-0.12640598  0.7189218   0.8370651  -1.2307327  -0.18723997 -0.557603
  0.6363

In [8]:
os.path.split(os.getcwd())

('/Users/maxwelladorsoo/Desktop/procupine', 'GenePT')